In [72]:
from models import Tournament
from utils import TournamentData, initialize_rating, ts_win_probability
import multiprocessing
from time import perf_counter
import trueskill as ts
from collections import defaultdict
from pprint import pprint
import pandas as pd
np = pd.np

In [6]:
data_path = '../../cwl-data/data/'
season = TournamentData(None,data_path,wwii=True)

In [79]:
team_players = t1.raw_data.sort_values('end_dt')[['team','player','win?','series id','end_dt']] \
    .groupby(['end_dt','series id','win?','team'])['player'].apply(set).reset_index() \
    .sort_values('end_dt')
team_players

,end_dt,series id,win?,team,player
0,2017-12-08 20:12:08,pool-D-5,L,Evil Geniuses,"{Apathy, Enable, Aches, Nameless}"
1,2017-12-08 20:12:08,pool-D-5,W,eUnited,"{Silly, Prestinni, Clayster, Arcitys}"
2,2017-12-08 20:14:26,pool-B-4,L,Ground Zero,"{Fero, Study, Blazt, Parasite}"
3,2017-12-08 20:14:26,pool-B-4,W,Team Vitality,"{Riskin, Wailers, Malls, Zayrox}"
4,2017-12-08 20:15:58,pool-B-2,L,Epsilon,"{Dqvee, Hawqeh, Vortex, Nathan}"
5,2017-12-08 20:15:58,pool-B-2,W,Luminosity Gaming,"{Octane, Slacked, JKap, John}"
6,2017-12-08 20:23:30,pool-D-1,L,Ghost Gaming,"{Maux, LlamaGod, Lacefield, SpaceLy}"
7,2017-12-08 20:23:30,pool-D-1,W,Splyce,"{Jurd, Bance, Tommey, MadCat}"
9,2017-12-08 20:29:00,pool-B-2,W,Luminosity Gaming,"{Octane, Slacked, JKap, John}"
8,2017-12-08 20:29:00,pool-B-2,L,Epsilon,"{Dqvee, Hawqeh, Vortex, Nathan}"


In [97]:
ordered_series = t1.raw_data[['series id','end_dt']] \
            .groupby('series id',as_index=False)['end_dt'].max() \
            .sort_values('end_dt').reset_index()
ordered_series['series id'].unique()

array(['pool-B-2', 'pool-D-1', 'pool-D-5', 'pool-B-4', 'pool-C-0',
       'pool-A-0', 'pool-C-7', 'pool-A-7', 'pool-B-0', 'pool-B-7',
       'pool-D-4', 'pool-D-2', 'pool-C-1', 'pool-C-5', 'pool-A-5',
       'pool-A-1', 'pool-D-0', 'pool-B-5', 'pool-D-7', 'pool-B-1',
       'pool-A-2', 'pool-C-2', 'pool-C-4', 'pool-A-4', 'pool-B-9',
       'pool-D-8', 'pool-A-8', 'pool-C-9', 'pool-D-3', 'pool-B-6',
       'pool-A-9', 'pool-C-6', 'pool-B-3', 'pool-A-3', 'pool-D-6',
       'pool-C-3', 'pool-D-9', 'pool-A-6', 'pool-B-8', 'pool-C-8',
       'champ-losers-1-0', 'champ-losers-1-2', 'champ-losers-1-3',
       'champ-losers-1-1', 'champ-losers-2-2', 'champ-losers-2-3',
       'champ-losers-2-0', 'champ-winners-1-3', 'champ-winners-1-2',
       'champ-losers-2-1', 'champ-losers-3-3', 'champ-winners-1-1',
       'champ-losers-3-0', 'champ-losers-3-2', 'champ-winners-1-0',
       'champ-losers-3-1', 'champ-losers-4-3', 'champ-winners-2-1',
       'champ-losers-4-2', 'champ-losers-4-0', 'champ-los

In [91]:
###
# build_for_team
###
tourney = []
series = {'data':{}}
sids = team_players['series id'].unique()
for s in sids: 
    sdata = team_players[team_players['series id']==s]
    series['id'] = s
    series['ended_at'] = sdata['end_dt'].max()
    score = np.unique(sdata[sdata['win?']=='W']['team'],return_counts=True)
    series['data'] = {
        'winner' : {
            'team' : score[0][np.argmax(score[1])],
            'score' : max(score[1]),
            'players': set.union(*sdata[sdata['team']==score[0][np.argmax(score[1])]].player)
        },
        'loser' : {
            'team' : score[0][np.argmin(score[1])],
            'score' : min(score[1]),
            'players' : set.union(*sdata[sdata['team']==score[0][np.argmin(score[1])]].player)
        }
        
    }
    tourney.append(series)
print(len(tourney))
pprint(sorted(tourney, key=lambda k: k['ended_at'], reverse=True)[0])

70
{'data': {'loser': {'players': {'Tommey', 'Bance', 'Jurd', 'MadCat'},
                    'score': 2,
                    'team': 'Splyce'},
          'winner': {'players': {'Theory', 'Kenny', 'Chino', 'Accuracy'},
                     'score': 3,
                     'team': 'Team Kaliber'}},
 'ended_at': Timestamp('2017-12-11 06:40:39'),
 'id': 'champ-grand-finals-0'}


In [13]:
tourney[:5]

[{'data': {'loser': {'team': 'Evil Geniuses',
    'players': ('Aches', 'Apathy', 'Enable', 'Nameless')},
   'winner': {'team': 'eUnited',
    'players': ('Arcitys', 'Clayster', 'Prestinni', 'Silly')}},
  'id': 'pool-D-5',
  'ended_at': Timestamp('2017-12-08 20:12:08')},
 {'data': {'loser': {'team': 'Ground Zero',
    'players': ('Fero', 'Parasite', 'Blazt', 'Study')},
   'winner': {'team': 'Team Vitality',
    'players': ('Zayrox', 'Wailers', 'Riskin', 'Malls')}},
  'id': 'pool-B-4',
  'ended_at': Timestamp('2017-12-08 20:14:26')},
 {'data': {'loser': {'team': 'Epsilon',
    'players': ('Hawqeh', 'Nathan', 'Dqvee', 'Vortex')},
   'winner': {'team': 'Luminosity Gaming',
    'players': ('John', 'Slacked', 'Octane', 'JKap')}},
  'id': 'pool-B-2',
  'ended_at': Timestamp('2017-12-08 20:15:58')},
 {'data': {'loser': {'team': 'Ghost Gaming',
    'players': ('Lacefield', 'LlamaGod', 'Maux', 'SpaceLy')},
   'winner': {'team': 'Splyce',
    'players': ('Bance', 'Jurd', 'MadCat', 'Tommey')}},
  

In [12]:
###
# set_initial_player_ratings
###
names = t1.raw_data.player.unique()
ratings = np.full((1,len(names)),1500)[0]

In [13]:
players = pd.DataFrame(columns=['player','rating'],data=np.array([names,ratings]).T)

In [14]:
names

array(['Arcitys', 'Clayster', 'Prestinni', 'Silly', 'Aches', 'Apathy',
       'Enable', 'Nameless', 'Blazt', 'Fero', 'Parasite', 'Study',
       'Malls', 'Riskin', 'Wailers', 'Zayrox', 'Dqvee', 'Hawqeh',
       'Nathan', 'Vortex', 'JKap', 'John', 'Octane', 'Slacked',
       'Lacefield', 'LlamaGod', 'Maux', 'SpaceLy', 'Bance', 'Jurd',
       'MadCat', 'Tommey', 'Joee', 'Joshh', 'Rated', 'Zer0', 'Accuracy',
       'Chino', 'Kenny', 'Theory', 'Bevils', 'Dashy', 'Decemate',
       'General', 'Crimsix', 'Formal', 'Karma', 'Scump', 'Buzzo', 'Denz',
       'Fighta', 'Shockz', 'Aqua', 'Felony', 'Loony', 'TJHaly', 'Classic',
       'Huke', 'Slasher', 'Temp', 'Moose', 'Seany', 'Skrapz', 'Wuskin',
       'Believe', 'Mayhem', 'Nova', 'Spoof', 'Attach', 'Priestah',
       'Replays', 'Zoomaa', 'Assault', 'Faccento', 'Gunless', 'Saints',
       'Censor', 'Methodz', 'Nagafen', 'Ricky', 'Dimi', 'Nelson', 'TcM',
       'Twizz', 'Goonjar', 'Proto', 'Royalty', 'Xotic', 'GodRx', 'Johnny',
       'Pandur', 

In [133]:
import trueskill as ts

In [141]:
t1 = (
    ts.Rating(mu=1500,sigma=100),
    ts.Rating(mu=1550,sigma=100),
    ts.Rating(mu=1500,sigma=100),
    ts.Rating(mu=1550,sigma=100),
)
t2 = (
    ts.Rating(mu=1550,sigma=100),
    ts.Rating(mu=1600,sigma=100),
    ts.Rating(mu=1550,sigma=100),
    ts.Rating(mu=1600,sigma=100),
)
ts.rate([t1,t2])

[(trueskill.Rating(mu=1545.902, sigma=95.116),
  trueskill.Rating(mu=1595.902, sigma=95.116),
  trueskill.Rating(mu=1545.902, sigma=95.116),
  trueskill.Rating(mu=1595.902, sigma=95.116)),
 (trueskill.Rating(mu=1504.098, sigma=95.116),
  trueskill.Rating(mu=1554.098, sigma=95.116),
  trueskill.Rating(mu=1504.098, sigma=95.116),
  trueskill.Rating(mu=1554.098, sigma=95.116))]